In [3]:
#!/usr/bin/env python
# coding: utf-8

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lower, when, to_timestamp, date_format,
    sum as spark_sum, coalesce, lit
)
from functools import reduce

# -----------------------
# 1. Start Spark session
# -----------------------
spark = SparkSession.builder.appName("MonthlySentimentScoresCombined").getOrCreate()

# -----------------------
# 2. Read cleaned CSV
#    (change this to your real cleaned file path)
# -----------------------
input_path = "output_clean.csv"

df = (
    spark.read
        .option("header", "true")
        .csv(input_path)
)

# Optional: restrict to English tweets
df = df.filter(col("tweet_language") == "en")

# Cast numeric columns & fill nulls with 0
df = (
    df
    .withColumn("n_replies",  coalesce(col("n_replies").cast("double"), lit(0.0)))
    .withColumn("n_likes",    coalesce(col("n_likes").cast("double"), lit(0.0)))
    .withColumn("n_retweets", coalesce(col("n_retweets").cast("double"), lit(0.0)))
)

# -----------------------
# 3. Define keyword / emoji lists (roots for stemming)
# -----------------------

def roots_to_regex(roots):
    # e.g. "surg" -> r"\bsurg\w*\b" to catch surge/surging/surged/etc
    return [rf"\b{root}\w*\b" for root in roots]

# Crypto + general positive sentiment roots
positive_roots = [
    # market / crypto bullish
    "bull", "bullish", "moon", "pump", "pamp", "breakout", "break",
    "surg", "rally", "spik", "rip", "rebound", "bounc",
    "green", "uptrend", "upside", "support", "hodl", "moonshot",
    "ath", "ripp", "moonboy", "moonboi",

    # profit / gains
    "gain", "profit", "win", "winn", "bagged", "baggin",
    "lucrativ", "reward", "windfall",

    # general good sentiment
    "good", "great", "awesom", "amaz", "fantast", "bullrun",
    "optimist"
]

positive_phrases = [
    "to the moon",
    "all time high",
    "new ath",
    "buy the dip",
    "strong support",
    "bull market",
    "mass adoption",
    "breakout above"
]

positive_emojis = [
    "😀", "😃", "😄", "😁", "😊", "😍", "🤩", "😂", "🤣",
    "🔥", "✨", "🚀", "📈", "💰", "💎", "🏆", "😎"
]

# Crypto + general negative sentiment roots
negative_roots = [
    # market / crypto bearish
    "bear", "bearish", "dump", "crash", "rekt",
    "dip", "selloff", "sell-off", "sell off", "bleed", "plummet",
    "liquidat", "downtrend", "downside",
    "resist", "overbought", "bubble",

    # scam / fraud / fear
    "scam", "fraud", "ponzi", "rugpull", "rug", "shitcoin",
    "fear", "panic", "panick",

    # general bad sentiment
    "bad", "terribl", "awful", "horribl", "suck", "loser",
    "loss", "losing", "weak", "trash", "garbage", "worst",
    "hate", "angry"
]

negative_phrases = [
    "all my money is gone",
    "rekt af",
    "got rekt",
    "paper hands",
    "bear market",
    "exit scam"
]

negative_emojis = [
    "😡", "😠", "😢", "😭", "😞", "😔",
    "💀", "☠", "📉", "🩸", "😰", "😱"
]

# -----------------------
# 4. Build detection conditions (roots + phrases + emojis)
# -----------------------

text_lower = lower(col("text"))

positive_regexes = roots_to_regex([r.lower() for r in positive_roots])
negative_regexes = roots_to_regex([r.lower() for r in negative_roots])

pos_conditions = (
    [text_lower.rlike(regex) for regex in positive_regexes] +
    [text_lower.contains(phrase.lower()) for phrase in positive_phrases] +
    [col("text").contains(e) for e in positive_emojis]
)

neg_conditions = (
    [text_lower.rlike(regex) for regex in negative_regexes] +
    [text_lower.contains(phrase.lower()) for phrase in negative_phrases] +
    [col("text").contains(e) for e in negative_emojis]
)

pos_cond = reduce(lambda a, b: a | b, pos_conditions) if pos_conditions else lit(False)
neg_cond = reduce(lambda a, b: a | b, neg_conditions) if neg_conditions else lit(False)

# -----------------------
# 5. Add month + weights + tweet-level scores
# -----------------------

# Parse date like "2019-05-27 11:49:18+00:00"
df = df.withColumn(
    "timestamp",
    to_timestamp(col("date"), "yyyy-MM-dd HH:mm:ssXXX")
)

df = df.withColumn(
    "month",
    date_format(col("timestamp"), "yyyy-MM")  # "YYYY-MM"
)

# weight = (n_replies + n_likes + n_retweets)/100 + 0.1
df = df.withColumn(
    "base_weight",
    (col("n_replies") + col("n_likes") + col("n_retweets")) / 100.0 + lit(0.1)
)

df = (
    df
    .withColumn("is_positive", pos_cond)
    .withColumn("is_negative", neg_cond)
)

df = (
    df
    .withColumn(
        "positive_score",
        when(col("is_positive"), col("base_weight")).otherwise(lit(0.0))
    )
    .withColumn(
        "negative_score",
        when(col("is_negative"), col("base_weight")).otherwise(lit(0.0))
    )
)

# -----------------------
# 6. Aggregate by month into ONE DataFrame
# -----------------------

monthly_sentiment = (
    df.groupBy("month")
      .agg(
          spark_sum("positive_score").alias("positive_score"),
          spark_sum(when(col("is_positive"), lit(1)).otherwise(lit(0))).alias("positive_tweet_count"),
          spark_sum("negative_score").alias("negative_score"),
          spark_sum(when(col("is_negative"), lit(1)).otherwise(lit(0))).alias("negative_tweet_count")
      )
      .orderBy("month")
)

# -----------------------
# 7. Write a single CSV (one output folder/file)
# -----------------------

output_path = "monthly_sentiment"

(
    monthly_sentiment
    .coalesce(1)            # force a single CSV file
    .write
    .mode("overwrite")
    .option("header", True)
    .csv(output_path)
)

spark.stop()
